In [1]:
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

# Rumus Petrofisika 

In [2]:
# Rumus Vshale
def vshale(gr,grmin,grmax):
    formula = (gr - grmin)/(grmax - grmin)
    formula = np.where(formula>1, 1, formula)
    formula = np.where(formula<0, 0, formula)
    return formula
# Rumus Neutron
def phin(nphi):
    formula = nphi
    return formula
    
# Rumus Porositas Denistias (PHID)
def phid(rhob, rhoma, rhofl):
    formula = (rhoma - rhob)/(rhoma - rhofl)
    formula = np.where(formula>1, 1, formula)
    formula = np.where(formula<0, 0, formula)
    return formula

# Rumus Porositas Total(PHIT)
def phit(phid, phin):
    phid2 = phid**2
    phin2 = phin**2
    formula = np.power((phid2+phin2)/2, 0.5)
    formula = np.where(formula>1, 1, formula)
    formula = np.where(formula<0, 0, formula)
    return formula

# Rumus Porositas efektif (PHIE)    
def phie(phit, vshale, phish):
    formula = phit - (vshale * phish)
    formula = np.where(formula>1, 1, formula)
    formula = np.where(formula<0, 0, formula)
    return formula


# Rumus Saturasi Air (SW) 
def sw(rt, phie, rw, a, m, n):
    top = a*rw
    bot = (phie**m) * rt
    sw_ = top / bot
    sw_ = np.power(sw_, 1/n)
    formula = sw_
    formula = np.where(formula>1, 1, formula)
    formula = np.where(formula<0, 0, formula)
    return formula


# Penggabungan data dan Perbaikan nama sumur sesuai data base

#### Memperbaiki Nama Sumur 

In [3]:
def word_to_vector(word):

     vec = [0]*len('abcdefghijklmnopqrstuvwxyz-''.()#&1234567890')
     for num, x in enumerate('abcdefghijklmnopqrstuvwxyz-''.()#&1234567890'):
         val = word.lower()
         val = val.count(x)
         vec[num] = val 
     return np.array(vec)

#### Mengambil data sumur dan marker

In [4]:
marker = pd.read_excel('lumpsum.xls')
wells = glob.glob('ARD/*')

In [5]:
len(marker)

611

In [6]:
len(wells)

431

In [7]:
marker.head(100)

,Well,ZoneName,Long,Lat,Type,Units,Top,Bottom,Gross,Net,...,AvGMIN,AvNWCL,AvRWCL,AvRw,AvTemp,Phi*H,PhiSo*H,Dryclay,Salinity,who
0,7 C FARMS #1,Heebner,-98.21577,38.596616,MD,ft,2607.00,2722.32,115.32,115.32,...,32.997,0.334,2.566,0.063,98.129,7.36,0.09,3.351351,95282.77387,gurning
1,A Goering #1,Heebner,-97.540324,38.335513,MD,ft,2081.14,2326.00,244.86,244.86,...,32.995,0.342,2.560,0.069,68.658,23.36,0.12,3.370821,122766.78410,gurning
2,Alan No.C-1,Heebner,-98.376504,38.164348,MD,ft,2914.00,3086.52,172.52,172.52,...,41.983,0.334,2.581,0.134,102.931,5.27,0.06,3.373874,38425.04008,gurning
3,Amstutz #1-8,Heebner,-98.344236,38.407085,MD,ft,2748.00,2901.00,153.00,153.00,...,37.984,0.294,2.601,0.102,100.416,8.28,0.71,3.267705,53759.20999,gurning
4,Ankerholz A No.1,Heebner,-98.399422,38.412072,MD,ft,2794.00,2937.00,143.00,143.00,...,39.986,0.320,2.585,0.132,101.002,4.67,0.26,3.330882,39836.00114,gurning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Long Land #1,Heebner,-98.29024,38.417676,MD,ft,2744.25,2906.00,161.75,161.75,...,29.989,0.317,2.567,0.035,100.425,5.03,0.28,3.294290,181961.94920,gurning
96,Long No. 1-13,Heebner,-98.370883,38.400607,MD,ft,2785.00,2944.00,159.00,159.00,...,41.983,0.356,2.575,0.031,100.988,5.22,0.70,3.445652,207796.33770,gurning
97,Lovett No.7,Heebner,-97.497934,38.395089,MD,ft,2074.25,2307.50,233.25,233.25,...,32.995,0.343,2.539,0.180,91.349,21.23,0.01,3.342466,30927.55145,gurning
98,LUDWICK #1,Heebner,-98.398599,38.359788,MD,ft,2826.00,2980.00,154.00,154.00,...,35.985,0.301,2.590,0.102,101.539,5.09,0.12,3.274678,53164.64442,gurning


In [8]:
well_marker = marker.Well.unique()
wells_data = [well.replace('ARD\\','').replace('.csv', '') for well in wells]
# menghilangkan tulisan CSV

In [9]:
len(marker.Well.unique())


306

In [10]:
penggabungan = {}
for w1 in well_marker:
    container = []
    for w2 in wells_data:
        w1v   = word_to_vector(w1)
        w2v   = word_to_vector(w2)
        diff = sum(np.abs(w1v - w2v)) 
        container.append(diff)
    loc = np.argmin(container)
    loc = wells_data[loc]
    penggabungan[w1] = loc+'.csv'

In [11]:
marker['well_data'] = [penggabungan[well] for well in marker.Well.values]
# membuat column baru dengan well data yang sudah di pairs

In [12]:
marker.Well.nunique()

306

# Tahap Perhitungan

In [13]:
sumur = marker.well_data.unique()

In [14]:
for well in sumur:

    try:

        temp = marker[marker['well_data'] == well]
        well_data = pd.read_csv('ARD/'+well).replace(-999.25, np.nan)

        dept = well_data['DEPT'].values

        dummy_grmin = np.array([np.nan]*len(dept))
        dummy_grmax = np.array([np.nan]*len(dept))
        dummy_phish = np.array([np.nan]*len(dept))
        dummy_rw = np.array([np.nan]*len(dept))

        for zone in temp.values:
            top = zone[6]
            bot = zone[7]
            grmin  = zone[15]
            grmax = zone[14]
            phish  = zone[16]
            rw  = zone[18]

            dummy_grmin = np.where((dept>top)&(dept<bot),grmin, dummy_grmin)
            dummy_grmax = np.where((dept>top)&(dept<bot),grmax, dummy_grmax)
            dummy_phish = np.where((dept>top)&(dept<bot),phish, dummy_phish)
            dummy_rw = np.where((dept>top)&(dept<bot),rw, dummy_rw)

        well_data['GRMin'] = dummy_grmin
        well_data['GRMax'] = dummy_grmax
        well_data['PHISH'] = dummy_phish
        well_data['Rw'] = dummy_rw

        well_data['Vshale'] = vshale(well_data['GR'], well_data['GRMin'], well_data['GRMax'])
        well_data['PHID'] = phid(well_data['RHOB'], 2.65, 1.1)
        well_data['PHIN'] = phin(well_data['NPHI'])
        well_data['PHIT'] = phit(well_data['PHID'], well_data['PHIN'])
        well_data['PHIE'] = phie(well_data['PHIT'], well_data['Vshale'], well_data['PHISH'])
        well_data['Sw'] = sw(well_data['RT'], well_data['PHIE'], well_data['Rw'], 1,2,2)

        well_data.to_csv('calculated/'+well, index=False)
        print('we are successfully calculating for well: ', well)
    except Exception:
        print('!!!!!!!!!! failed to calculate on well:', well)
        print(well_data.columns)

we are successfully calculating for well:  7 C Farms 1.csv
we are successfully calculating for well:  A. Goering 1.csv
we are successfully calculating for well:  Alan C-1.csv
we are successfully calculating for well:  Amstutz 1-8.csv
we are successfully calculating for well:  Ankerholz 'A' 1.csv
we are successfully calculating for well:  Appleton 5.csv
we are successfully calculating for well:  B.B. Ainsworth 13.csv
we are successfully calculating for well:  Beagley 1.csv
we are successfully calculating for well:  Behnke 1-15.csv
we are successfully calculating for well:  Berry 6.csv
we are successfully calculating for well:  Berry 5.csv
we are successfully calculating for well:  Blackhall 5.csv
we are successfully calculating for well:  BNE 19-22.csv
we are successfully calculating for well:  BONNEWELL-CARTER UNIT 18-1.csv
we are successfully calculating for well:  Boye 1-16.csv
we are successfully calculating for well:  BOYE 1-9.csv
!!!!!!!!!! failed to calculate on well: Bredfeldt-B

In [15]:
wellww = glob.glob('calculated/*')

In [16]:
len(wellww)

277

In [17]:
test=pd.DataFrame(wellww)

In [18]:
dataset

NameError: name 'dataset' is not defined